# User Guide

All `xarray_jsonschema` schema extend the base  {py:class}`XarraySchema` class and are functionally equivalent. But it makes sense to separate them into 2 groups, based on the way they are composed.

1. [Component schemas](#component-schemas)
2. [Container schemas](#container-schemas)

:::{toctree}
:titlesonly:
:maxdepth: 2
:hidden:

examples/index.md
:::

In [1]:
import numpy as np
import xarray as xr
import xarray_jsonschema as xjs

# Example data array
da = xr.DataArray(
    name='foo',
    dims=('x', 'y'),
    data=np.reshape(np.ones(4, dtype='i4'), (2, 2)),
)

## Component Schemas

Component schema specify the core components of the `xarray` [data model](https://docs.xarray.dev/en/stable/user-guide/index.html), things like `dtype`, `dims`, `name`, `shape`, etc.

In [2]:
name_schema = xjs.NameSchema('foo')
shape_schema = xjs.ShapeSchema((2, None))  # ``None`` is used as a wildcard

Schema can compose other schema to specify more complex constraints.

In [3]:
dims_schema = xjs.DimsSchema(('x', xjs.NameSchema('^[a-z]$', regex=True)))

Component schema are fully fledged schema that can be used as standalone validators.

In [4]:
dims_schema.validate(da)

But typically component schema are used to build [container schemas](#container-schema).

## Container Schemas

Container schemas are schema that compose component schema to build schema that can fully specify `xarray` [data structures](https://docs.xarray.dev/en/stable/user-guide/data-structures.html). For simple constraints, you can pass Python types and they will be converted to the appropriate component schema under the hood.

In [5]:
da_schema = xjs.DataArraySchema(
    dims=dims_schema,
    name=name_schema,
    shape=shape_schema,
    # Will converted to `DTypeSchema` internally.
    dtype=np.integer,
)

## Validation

All schema objects expose the {py:meth}`~.XarraySchema.validate` method. The `validate` method accepts an `xarray` object. Internally, the object is **always** converted to a Python dictionary with:

```python
instance = da.to_dict(data=False)
```

If validation is successful, no value is returned. Otherwise, a {py:class}`jsonschema.exceptions.ValidationError` is raised.

In [6]:
try:
    da_schema.validate(da)
except xjs.ValidationError as error:
    print(error.message)


'int64' was expected


## JSON Schema

Use the {py:attr}`~.XarraySchema.json` property to view the JSON Schema.

In [7]:
da_schema.json

{'type': 'object',
 'properties': {'dims': {'type': 'array',
   'items': False,
   'prefixItems': [{'const': 'x'}, {'type': 'string', 'pattern': '^[a-z]$'}],
   'minItems': 2},
  'dtype': {'const': 'int64'},
  'shape': {'type': 'array',
   'items': False,
   'prefixItems': [{'const': 2}, {'type': 'integer'}],
   'minItems': 2},
  'name': {'const': 'foo'}}}

Or use the {py:meth}`~.XarraySchema.dumps` method to so serialize the schema to a JSON formatted string.

In [8]:
print(da_schema.dumps(indent=2))

{
  "type": "object",
  "properties": {
    "dims": {
      "type": "array",
      "items": false,
      "prefixItems": [
        {
          "const": "x"
        },
        {
          "type": "string",
          "pattern": "^[a-z]$"
        }
      ],
      "minItems": 2
    },
    "dtype": {
      "const": "int64"
    },
    "shape": {
      "type": "array",
      "items": false,
      "prefixItems": [
        {
          "const": 2
        },
        {
          "type": "integer"
        }
      ],
      "minItems": 2
    },
    "name": {
      "const": "foo"
    }
  }
}


View the [Examples](examples) for more detailed usage and examples.